In [54]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow_datasets as tfds

import numpy as np
import os
import PIL
import PIL.Image
import glob

import cv2

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [55]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [56]:
ds = image_dataset_from_directory(
'tiny_imagenet/train',
image_size=(64,64),
batch_size=32)

Found 100000 files belonging to 200 classes.


In [57]:

split = int(len(ds)*.8)
print(split)

2500


In [58]:
train_size = int(len(ds)*.8)
val_size = len(ds) - train_size
ds_train = ds.take(train_size)
ds_val = ds.skip(split).take(val_size)

In [64]:
conv_base = keras.applications.vgg19.VGG19(
weights="imagenet",
input_shape=(64, 64, 3),
include_top=False)

conv_base.trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [65]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)

In [81]:
model = keras.Sequential([
    data_augmentation,
    conv_base,
    layers.Flatten(),
    layers.Dense(units=4096, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=2048, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=1024, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=200, activation="softmax"),
])

In [82]:
opt = tf.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [78]:
# inputs = keras.Input(shape=(64, 64, 3))
# x = data_augmentation(inputs)
# x = keras.applications.vgg19.preprocess_input(x)
# x = conv_base(x)
# x = layers.Flatten()(x)
# x = layers.Dense(512)(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(256)(x)
# x = layers.Dropout(0.3)(x)
# outputs = layers.Dense(200, activation="softmax")(x)
# model = keras.Model(inputs, outputs)
# model.compile(loss="SparseCategoricalCrossentropy",
# optimizer="adam",
# metrics=["accuracy"])

In [88]:
inputs = keras.Input(shape=(64,64,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="SparseCategoricalCrossentropy",
optimizer="adam",
metrics=["accuracy"])



In [89]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_without_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [91]:
history = model.fit(
ds_train,
epochs=80,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/80
2500/2500 [==============================] - 68s 27ms/step - loss: 3.3232 - accuracy: 0.2394 - val_loss: 3.4992 - val_accuracy: 0.2300
Epoch 2/80
2500/2500 [==============================] - 69s 28ms/step - loss: 3.3028 - accuracy: 0.2449 - val_loss: 3.3814 - val_accuracy: 0.2404
Epoch 3/80
2500/2500 [==============================] - 69s 28ms/step - loss: 3.2988 - accuracy: 0.2448 - val_loss: 3.4429 - val_accuracy: 0.2380
Epoch 4/80
2500/2500 [==============================] - 69s 28ms/step - loss: 3.2794 - accuracy: 0.2500 - val_loss: 3.4304 - val_accuracy: 0.2448
Epoch 5/80
2500/2500 [==============================] - 69s 28ms/step - loss: 3.2678 - accuracy: 0.2506 - val_loss: 3.4403 - val_accuracy: 0.2397
Epoch 6/80
2500/2500 [==============================] - 68s 27ms/step - loss: 3.2509 - accuracy: 0.2542 - val_loss: 3.3169 - val_accuracy: 0.2566
Epoch 7/80
1243/2500 [=============>................] - ETA: 33s - loss: 3.2519 - accuracy: 0.2527